In [2]:
import pandas as pd
import numpy as np
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError

In [54]:
y_train = np.repeat(758.88,19)
y_true = np.array([741.84,742.58,743.62,726.39,714.47,716.03,726.07,700.56,714.72,694.45,701.79,698.45,706.59,725.25,711.67,713.04,699.99,730.96,742.95])
y_pred = np.repeat(601.55,19)
mase = MeanAbsoluteScaledError()
mase(y_true, y_pred)

ValueError: mean_absolute_scaled_error requires `y_train`. Pass `y_train` as a keyword argument when calling the metric.

In [52]:
len(y_pred)

19

In [47]:
y_train

array([758.88])

In [4]:
train = pd.read_csv('Data/sales_train_evaluation.csv')
cal = pd.read_csv('Data/calendar.csv')
prices = pd.read_csv('Data/sell_prices.csv')

In [9]:
train_naive = train.iloc[:,:1919]

In [12]:
train_naive_t = train_naive.iloc[0:3,:]

In [13]:
train_naive_t

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1


In [77]:
(train_naive_t[train_naive_t.columns[7]] - train_naive_t[train_naive_t.columns[6]])**2

0    0
1    0
2    0
dtype: int64

In [33]:
train.shape[0]

30490

In [86]:
# range starting from 6 is for the initial 6 non-demand columns
def add_naive_rmse(df,headers):
    naive_rmse = pd.DataFrame(data={'naive_rmse':list(np.repeat(0,df.shape[0]))})
    #print(naive_rmse)
    for i in range(headers,len(df.columns)-1):
        new_naive_rmse = (df[df.columns[i+1]] - df[df.columns[i]])**2
        naive_rmse['naive_rmse'] += new_naive_rmse
        #print(naive_rmse)
    naive_rmse = naive_rmse**.5
    return naive_rmse
    

In [87]:
naive_rmse = add_naive_rmse(train_naive_t,6)

In [90]:
test_df = pd.DataFrame(data={'day1':[4],'day2':[8],'day3':[10],'day4':[13]})

In [91]:
test_df

,day1,day2,day3,day4
0,4,8,10,13


In [92]:
add_naive_rmse(test_df,0)

,naive_rmse
0,5.385165


In [94]:
(16+4+9)**.5

5.385164807134504

In [96]:
states = train.iloc[:,5:].groupby('state_id').sum()

In [97]:
states

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
state_id,,,,,,,,,,,,,,,,,,,,,
CA,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,18471,23037,24704,17721,16150,15678,16297,17430,23103,24644
TX,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,12320,12366,16249,12228,11370,10375,9162,12303,13681,14815
WI,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,13282,18905,18968,12413,11257,11043,11504,12819,14734,14879


In [110]:
selection = [4] + [i for i in range(6,1947)]
stores = train.iloc[:,selection].groupby('store_id').sum()

In [111]:
stores

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,4951,6245,6707,4568,3949,3995,4136,4433,5764,6289
CA_2,3494,3046,2121,2324,1942,2288,2629,3729,2957,2218,...,4839,6507,6710,4238,3963,3789,4037,4751,7120,6614
CA_3,4739,4827,3785,4232,3817,4369,4703,5456,5581,4912,...,6046,7029,7969,5891,5523,5375,5580,5542,7073,8144
CA_4,1625,1777,1386,1440,1536,1389,1469,1988,1818,1535,...,2635,3256,3318,3024,2715,2519,2544,2704,3146,3597
TX_1,2556,2687,1822,2258,1694,2734,1691,2820,2887,2174,...,3610,3787,5000,3708,3310,3147,2748,3664,4167,4624
TX_2,3852,3937,2731,2954,2492,3439,2588,3772,3657,2932,...,4415,4424,5866,4179,4008,3518,3126,4249,4802,5217
TX_3,3030,3006,2225,2169,1726,2833,1947,2848,2832,2213,...,4295,4155,5383,4341,4052,3710,3288,4390,4712,4974
WI_1,2704,2194,1562,1251,2,2049,2815,3248,1674,1355,...,3978,5527,5488,3505,3323,3242,3478,3813,5002,5054
WI_2,2256,1922,2018,2522,1175,2244,2232,2643,2140,1836,...,5317,7704,7586,5011,4628,4533,4628,4880,5213,5068


In [109]:
selection = [3] + [i for i in range(6,1947)]
category = train.iloc[:,selection].groupby('cat_id').sum()

In [112]:
category

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
cat_id,,,,,,,,,,,,,,,,,,,,,
FOODS,23178,22758,17174,18878,14603,22093,20490,27751,24862,18901,...,30107,36576,41447,29158,26148,24790,24737,28136,33599,35967
HOBBIES,3764,3357,2682,2669,1814,3220,2944,3986,2899,2615,...,3871,4893,5016,3528,3629,3740,3475,4143,5333,5280
HOUSEHOLD,5689,5634,3927,3865,2729,3898,4576,6195,4975,4056,...,10095,12839,13458,9676,9000,8566,8751,10273,12586,13091


In [113]:
selection = [2] + [i for i in range(6,1947)]
department = train.iloc[:,selection].groupby('dept_id').sum()

In [114]:
department

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1,2343,2216,1657,1508,1209,1897,1903,2235,1925,1586,...,4130,4257,4225,3264,3119,3073,3251,3735,4195,4031
FOODS_2,4094,4209,3174,3606,2869,4375,3349,4384,4273,3684,...,5842,7563,9069,6226,5311,4648,4551,4815,5940,6535
FOODS_3,16741,16333,12343,13764,10525,15821,15238,21132,18664,13631,...,20135,24756,28153,19668,17718,17069,16935,19586,23464,25401
HOBBIES_1,3610,3172,2497,2531,1714,3133,2855,3831,2741,2514,...,3422,4414,4464,3109,3142,3291,3027,3761,4832,4720
HOBBIES_2,154,185,185,138,100,87,89,155,158,101,...,449,479,552,419,487,449,448,382,501,560
HOUSEHOLD_1,4105,3858,2827,2732,1802,2664,3161,4334,3494,2857,...,7936,10042,10572,7609,6999,6641,6759,7974,9668,10165
HOUSEHOLD_2,1584,1776,1100,1133,927,1234,1415,1861,1481,1199,...,2159,2797,2886,2067,2001,1925,1992,2299,2918,2926


In [117]:
selection = [3,5] + [i for i in range(6,1947)]
state_cat = train.iloc[:,selection].groupby(['state_id','cat_id']).sum()

In [118]:
state_cat

d_1   d_2   d_3   d_4   d_5   d_6   d_7    d_8    d_9  \
state_id cat_id                                                               
CA       FOODS      10101  9862  6944  7864  7178  8256  9005  11870  10977   
         HOBBIES     1802  1561  1472  1405  1181  1459  1314   1986   1482   
         HOUSEHOLD   2292  2382  1692  1778  1566  1607  1932   2754   2237   
TX       FOODS       6853  7030  5124  5470  4602  7067  4671   7055   6920   
         HOBBIES      879   870   526   809   501   831   390    785    794   
         HOUSEHOLD   1706  1730  1128  1102   809  1108  1165   1600   1662   
WI       FOODS       6224  5866  5106  5544  2823  6770  6814   8826   6965   
         HOBBIES     1083   926   684   455   132   930  1240   1215    623   
         HOUSEHOLD   1691  1522  1107   985   354  1183  1479   1841   1076   

                    d_10  ...  d_1932  d_1933  d_1934  d_1935  d_1936  d_1937  \
state_id cat_id           ...                                                   
CA       FOODS      8637  ...   12151   14447   16197   11859   10759   10465   
         HOBBIES    1508  ...    1831    2638    2401    1554    1649    1606   
         HOUSEHOLD  1677  ...    4489    5952    6106    4308    3742    3607   
TX       FOODS      5505  ...    8369    8095   10663    7905    7164    6596   
         HOBBIES     524  ...    1028     933    1461    1205    1199    1056   
         HOUSEHOLD  1290  ...    2923    3338    4125    3118    3007    2723   
WI       FOODS      4759  ...    9587   14034   14587    9394    8225    7729   
         HOBBIES     583  ...    1012    1322    1154     769     781    1078   
         HOUSEHOLD  1089  ...    2683    3549    3227    2250    2251    2236   

                    d_1938  d_1939  d_1940  d_1941  
state_id cat_id                                     
CA       FOODS       10650   11354   14489   15970  
         HOBBIES      1735    1975    2701    2514  
         HOUSEHOLD    3912    4101    5913    6160  
TX       FOODS        6083    7965    8834    9500  
         HOBBIES       830    1132    1440    1543  
         HOUSEHOLD    2249    3206    3407    3772  
WI       FOODS        8004    8817   10276   10497  
         HOBBIES       910    1036    1192    1223  
         HOUSEHOLD    2590    2966    3266    3159  

[9 rows x 1941 columns]

In [120]:
selection = [2,5] + [i for i in range(6,1947)]
state_dept = train.iloc[:,selection].groupby(['state_id','dept_id']).sum()

In [121]:
state_dept

d_1   d_2   d_3   d_4   d_5   d_6   d_7   d_8   d_9  \
state_id dept_id                                                             
CA       FOODS_1      1157  1142   787   743   729   796   913  1036  1049   
         FOODS_2      1864  1987  1315  1452  1239  1427  1554  1962  1928   
         FOODS_3      7080  6733  4842  5669  5210  6033  6538  8872  8000   
         HOBBIES_1    1739  1502  1422  1352  1139  1427  1281  1920  1419   
         HOBBIES_2      63    59    50    53    42    32    33    66    63   
         HOUSEHOLD_1  1527  1515  1116  1148   935  1014  1224  1834  1496   
         HOUSEHOLD_2   765   867   576   630   631   593   708   920   741   
TX       FOODS_1       500   446   381   317   297   445   337   466   432   
         FOODS_2      1384  1426  1008  1316  1143  1826   917  1319  1280   
         FOODS_3      4969  5158  3735  3837  3162  4796  3417  5270  5208   
         HOBBIES_1     837   775   435   756   448   798   355   732   725   
         HOBBIES_2      42    95    91    53    53    33    35    53    69   
         HOUSEHOLD_1  1281  1261   845   820   598   801   843  1142  1200   
         HOUSEHOLD_2   425   469   283   282   211   307   322   458   462   
WI       FOODS_1       686   628   489   448   183   656   653   733   444   
         FOODS_2       846   796   851   838   487  1122   878  1103  1065   
         FOODS_3      4692  4442  3766  4258  2153  4992  5283  6990  5456   
         HOBBIES_1    1034   895   640   423   127   908  1219  1179   597   
         HOBBIES_2      49    31    44    32     5    22    21    36    26   
         HOUSEHOLD_1  1297  1082   866   764   269   849  1094  1358   798   
         HOUSEHOLD_2   394   440   241   221    85   334   385   483   278   

                      d_10  ...  d_1932  d_1933  d_1934  d_1935  d_1936  \
state_id dept_id            ...                                           
CA       FOODS_1       804  ...    1931    1854    1820    1408    1316   
         FOODS_2      1787  ...    2108    2396    3130    2203    1955   
         FOODS_3      6046  ...    8112   10197   11247    8248    7488   
         HOBBIES_1    1471  ...    1643    2386    2195    1415    1495   
         HOBBIES_2      37  ...     188     252     206     139     154   
         HOUSEHOLD_1  1089  ...    3378    4398    4598    3267    2799   
         HOUSEHOLD_2   588  ...    1111    1554    1508    1041     943   
TX       FOODS_1       368  ...    1022     924    1025     824     797   
         FOODS_2      1111  ...    1320    1339    1914    1521    1254   
         FOODS_3      4026  ...    6027    5832    7724    5560    5113   
         HOBBIES_1     496  ...     872     827    1253    1010     975   
         HOBBIES_2      28  ...     156     106     208     195     224   
         HOUSEHOLD_1   937  ...    2347    2717    3353    2515    2383   
         HOUSEHOLD_2   353  ...     576     621     772     603     624   
WI       FOODS_1       414  ...    1177    1479    1380    1032    1006   
         FOODS_2       786  ...    2414    3828    4025    2502    2102   
         FOODS_3      3559  ...    5996    8727    9182    5860    5117   
         HOBBIES_1     547  ...     907    1201    1016     684     672   
         HOBBIES_2      36  ...     105     121     138      85     109   
         HOUSEHOLD_1   831  ...    2211    2927    2621    1827    1817   
         HOUSEHOLD_2   258  ...     472     622     606     423     434   

                      d_1937  d_1938  d_1939  d_1940  d_1941  
state_id dept_id                                              
CA       FOODS_1        1374    1364    1631    2095    2041  
         FOODS_2        1736    1795    1805    2421    2888  
         FOODS_3        7355    7491    7918    9973   11041  
         HOBBIES_1      1460    1545    1816    2469    2248  
         HOBBIES_2       146     190     159     232     266  
         HOUSEHOLD_1    2663    2928    3045    4342    4638 

In [123]:
selection = [3,4] + [i for i in range(6,1947)]
store_cat = train.iloc[:,selection].groupby(['store_id','cat_id']).sum()

In [124]:
store_cat

d_1   d_2   d_3   d_4   d_5   d_6   d_7   d_8   d_9  \
store_id cat_id                                                            
CA_1     FOODS      3239  3137  2008  2258  2032  2407  2693  3962  3418   
         HOBBIES     556   498   415   392   268   467   346   769   432   
         HOUSEHOLD   542   520   393   401   330   402   411   706   490   
CA_2     FOODS      2193  1921  1289  1540  1278  1494  1785  2385  1827   
         HOBBIES     538   397   368   350   296   391   316   413   452   
         HOUSEHOLD   763   728   464   434   368   403   528   931   678   
CA_3     FOODS      3446  3535  2701  3064  2761  3340  3455  4068  4324   
         HOBBIES     550   430   438   424   364   390   493   525   375   
         HOUSEHOLD   743   862   646   744   692   639   755   863   882   
CA_4     FOODS      1223  1269   946  1002  1107  1015  1072  1455  1408   
         HOBBIES     158   236   251   239   253   211   159   279   223   
         HOUSEHOLD   244   272   189   199   176   163   238   254   187   
TX_1     FOODS      1781  1903  1377  1643  1329  2091  1247  2008  2061   
         HOBBIES     251   264   107   248   102   292    89   264   242   
         HOUSEHOLD   524   520   338   367   263   351   355   548   584   
TX_2     FOODS      2788  2870  2082  2180  1890  2696  1963  2890  2750   
         HOBBIES     385   359   229   359   257   318   162   270   302   
         HOUSEHOLD   679   708   420   415   345   425   463   612   605   
TX_3     FOODS      2284  2257  1665  1647  1383  2280  1461  2157  2109   
         HOBBIES     243   247   190   202   142   221   139   251   250   
         HOUSEHOLD   503   502   370   320   201   332   347   440   473   
WI_1     FOODS      1581  1327   977   935     2  1278  1671  2016  1086   
         HOBBIES     615   443   323   137     0   468   734   640   325   
         HOUSEHOLD   508   424   262   179     0   303   410   592   263   
WI_2     FOODS      1615  1433  1586  2013   967  1709  1668  2064  1786   
         HOBBIES     190   127   113   124    58   169   146   163    78   
         HOUSEHOLD   451   362   319   385   150   366   418   416   276   
WI_3     FOODS      3028  3106  2543  2596  1854  3783  3475  4746  4093   
         HOBBIES     278   356   248   194    74   293   360   412   220   
         HOUSEHOLD   732   736   526   421   204   514   651   833   537   

                    d_10  ...  d_1932  d_1933  d_1934  d_1935  d_1936  d_1937  \
store_id cat_id           ...                                                   
CA_1     FOODS      2446  ...    3365    4046    4717    3198    2794    2816   
         HOBBIES     384  ...     588     806     687     417     393     381   
         HOUSEHOLD   327  ...     998    1393    1303     953     762     798   
CA_2     FOODS      1355  ...    3255    3972    4285    2902    2649    2561   
         HOBBIES     403  ...     370     687     588     326     385     334   
         HOUSEHOLD   460  ...    1214    1848    1837    1010     929     894   
CA_3     FOODS      3718  ...    3864    4311    4980    3685    3491    3425   
         HOBBIES     478  ...     502     665     713     482     532     565   
         HOUSEHOLD   716  ...    1680    2053    2276    1724    1500    1385   
CA_4     FOODS      1118  ...    1667    2118    2215    2074    1825    1663   
         HOBBIES     243  ...     371     480     413     329     339     326   
         HOUSEHOLD   174  ...     597     658     690     621     551     530   
TX_1     FOODS      1579  ...    2403    2528    3136    2193    2026    1924   
         HOBBIES     127  ...     287     234     390     443     278     257   
         HOUSEHOLD   468  ...     920    1025    1474    1072    1006     966   
TX_2     FOODS      2189  ...    2964    2889    3887    2858    2614    2270   
         HOBBIES     273  ...     381     374     556     343     458     385   
         HOUSEHOLD   470  ...    1070    1161    

In [125]:
selection = [2,4] + [i for i in range(6,1947)]
store_dept = train.iloc[:,selection].groupby(['store_id','dept_id']).sum()

In [126]:
store_dept

d_1   d_2   d_3   d_4   d_5   d_6   d_7   d_8   d_9  \
store_id dept_id                                                             
CA_1     FOODS_1       297   284   214   175   182   191   224   263   245   
         FOODS_2       674   655   396   476   354   486   554   747   581   
         FOODS_3      2268  2198  1398  1607  1496  1730  1915  2952  2592   
         HOBBIES_1     528   489   409   383   263   453   339   750   425   
         HOBBIES_2      28     9     6     9     5    14     7    19     7   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
WI_3     FOODS_3      2293  2383  1841  1965  1427  2760  2727  3786  3179   
         HOBBIES_1     256   342   228   183    70   285   357   399   216   
         HOBBIES_2      22    14    20    11     4     8     3    13     4   
         HOUSEHOLD_1   584   541   420   327   151   392   492   647   418   
         HOUSEHOLD_2   148   195   106    94    53   122   159   186   119   

                      d_10  ...  d_1932  d_1933  d_1934  d_1935  d_1936  \
store_id dept_id            ...                                           
CA_1     FOODS_1       176  ...     412     435     415     277     302   
         FOODS_2       559  ...     545     611     781     555     476   
         FOODS_3      1711  ...    2408    3000    3521    2366    2016   
         HOBBIES_1     375  ...     546     732     631     389     347   
         HOBBIES_2       9  ...      42      74      56      28      46   
...                    ...  ...     ...     ...     ...     ...     ...   
WI_3     FOODS_3      1924  ...    2061    2815    3162    1952    1602   
         HOBBIES_1     222  ...     202     345     294     187     174   
         HOBBIES_2      21  ...      21      37      38      11      45   
         HOUSEHOLD_1   354  ...     608     937     874     626     597   
         HOUSEHOLD_2   108  ...     164     205     194     138     132   

                      d_1937  d_1938  d_1939  d_1940  d_1941  
store_id dept_id                                              
CA_1     FOODS_1         397     330     310     427     334  
         FOODS_2         427     411     434     627     719  
         FOODS_3        1992    2113    2249    2729    3274  
         HOBBIES_1       353     354     494     656     588  
         HOBBIES_2        28      52      45      63      58  
...                      ...     ...     ...     ...     ...  
WI_3     FOODS_3        1594    1627    2011    2360    2632  
         HOBBIES_1       287     220     242     273     305  
         HOBBIES_2        45      46      27      31      37  
         HOUSEHOLD_1     486     595     764     723     723  
         HOUSEHOLD_2     133     141     230     218     208  

[70 rows x 1941 columns]

In [127]:
selection = [1] + [i for i in range(6,1947)]
product = train.iloc[:,selection].groupby(['item_id']).sum()

In [128]:
product

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001,6,6,4,6,7,18,10,4,11,10,...,7,8,8,4,7,7,5,7,5,9
FOODS_1_002,4,5,7,4,3,4,1,7,2,4,...,5,8,7,2,6,5,0,6,6,4
FOODS_1_003,14,8,3,6,3,8,13,10,11,6,...,7,8,6,11,9,7,7,10,6,5
FOODS_1_004,0,0,0,0,0,0,0,0,0,0,...,115,104,107,81,73,62,71,75,83,93
FOODS_1_005,34,32,13,20,10,21,18,20,25,41,...,15,16,25,9,7,25,19,25,17,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512,5,4,1,3,2,4,2,8,7,5,...,5,7,11,4,8,3,4,5,4,7
HOUSEHOLD_2_513,0,0,0,0,0,0,0,0,0,0,...,4,0,5,3,6,3,0,9,3,4
HOUSEHOLD_2_514,4,8,2,1,1,2,3,8,2,1,...,2,2,2,6,1,0,1,4,3,3


In [130]:
selection = [1,5] + [i for i in range(6,1947)]
product_state = train.iloc[:,selection].groupby(['item_id','state_id']).sum()

In [131]:
product_state

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  d_1932  d_1933  d_1934  d_1935  d_1936  d_1937  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       5       5       3       1       1       2   
                TX        ...       1       1       2       2       5       1   
                WI        ...       1       2       3       1       1       4   
FOODS_1_002     CA        ...       3       4       6       2       1       2   
                TX        ...       1       1       1       0       2       1   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       1       0       0       0   
                WI        ...       0       0       1       4       1       1   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       3       1   
                TX        ...       0       1       0       1       0       0   
                WI        ...       0       1       1       1       0       0   

                          d_1938  d_1939  d_1940  d_1941  
item_id         state_id                                  
FOODS_1_001     CA             2       4       4       2  
                TX             2       3       0       3  
                WI             1       0       1       4  
FOODS_1_002     CA             0       4       4       4  
                TX             0       0       0       0  
...                          ...     ...     ...     ...  
HOUSEHOLD_2_515 TX             3       0       1       0  
                WI             0       0       0       1  
HOUSEHOLD_2_516 CA             1       1       3       0  
                TX             0       1       1       3  
                WI             0       1       0       0  

[9147 rows x 1941 columns]

In [132]:
total = train.iloc[:,6:].sum()

In [133]:
total

d_1       32631
d_2       31749
d_3       23783
d_4       25412
d_5       19146
          ...  
d_1937    37096
d_1938    36963
d_1939    42552
d_1940    51518
d_1941    54338
Length: 1941, dtype: int64